In [1]:
import pandas as pd
from sklearn.metrics import roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim 
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
import numpy as np
import os
from tqdm import tqdm

file = 'NA'

file_prefixes = ["smiles","DMPNN", "ecfp", "mfbert", "padel"]
file_extension = ".csv"

file_prefix = f"{file}_Feature_fusion/data_train"
file_extension = ".csv"

val_file_prefix = f"{file}_Feature_fusion/data_valid"
val_file_extension = ".csv"

start_index = 0
end_index = 4

auc_scores = []
acc_scores = []
f1_scores = []
recall_scores = []
mcc_scores = []


for i in tqdm(range(start_index, end_index + 1), desc="Processing files"):  # 使用 tqdm 添加进度条  
    combined_data_train = None
    combined_data_valid = None
    combined_data_test = None
    for file_feature in file_prefixes:
        #从randomi/中获取train，valid，test    
        
        file_path_train = file + "/" +'random' + str(i) + "/" + file_feature + "/" + "train" + file_extension
        data_train = pd.read_csv(file_path_train) 
                
        file_path_valid = file + "/" +'random' + str(i) + "/" + file_feature + "/" + "valid" + file_extension
        data_valid = pd.read_csv(file_path_valid) 

        file_path_test = file + "/" +'random' + str(i) + "/" + file_feature + "/" + "test" + file_extension
        data_test = pd.read_csv(file_path_test)   
        
        data_train = pd.read_csv(file_path_train)     
        data_valid = pd.read_csv(file_path_valid)   
        data_test  = pd.read_csv(file_path_test) 
        
        if combined_data_train is None:
            combined_data_train = data_train
        if combined_data_valid is None:
            combined_data_valid = data_valid
        if combined_data_test is None:   
            combined_data_test  = data_test
        else:
            combined_data_train = pd.concat([combined_data_train, data_train], axis=1)     
            combined_data_valid = pd.concat([combined_data_valid, data_valid], axis=1) 
            combined_data_test = pd.concat([combined_data_test, data_test], axis=1) 
    
    folder_path = f'{file}_Feature_fusion'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    combined_data_train.to_csv(f'{file}_Feature_fusion/data_train{i}.csv', index=False)
    combined_data_valid.to_csv(f'{file}_Feature_fusion/data_valid{i}.csv', index=False)
    combined_data_test.to_csv(f'{file}_Feature_fusion/data_test{i}.csv', index=False)


Processing files: 100%|██████████| 5/5 [00:05<00:00,  1.13s/it]


In [2]:
# 读取CSV文件
file_prefix = f"{file}_Feature_fusion/data_train"
file_extension = ".csv"

val_file_prefix = f"{file}_Feature_fusion/data_valid"
val_file_extension = ".csv"

for i in range(start_index, end_index+1):
    
    # 读取数据
    file_path = file_prefix + str(i) + file_extension
    data = pd.read_csv(file_path)
    null_indices = None  # 初始化 null_indices 为 None
    if data.isnull().values.any():  # 判断是否有空值
        print("数据中存在空值，空值坐标为：")
        null_indices = np.where(data.isnull())  # 获取空值的坐标
        if null_indices is not None:  # 再次判断 null_indices 是否有值
            for row_index, col_index in zip(null_indices[0], null_indices[1]):
                print(f"行 {row_index}，列 {col_index}")
    else:
        print("数据中没有空值")
    print(f"该文件的行数为：{data.shape[0]}，列数为：{data.shape[1]}")
    

数据中没有空值
该文件的行数为：816，列数为：2712
数据中没有空值
该文件的行数为：816，列数为：2712
数据中没有空值
该文件的行数为：816，列数为：2712
数据中没有空值
该文件的行数为：816，列数为：2712
数据中没有空值
该文件的行数为：816，列数为：2712


In [40]:
# 读取CSV文件
for i in range(start_index, end_index+1):
    
    # 读取数据
    file_path = file_prefix + str(i) + file_extension
    data = pd.read_csv(file_path)
    
    # 提取特征和标签
    x_input = data.drop(['SMILES'], axis=1).values
    y_output = data['activity'].values
    
        # 遍历数据，将 inf 赋值为 0
    x_input = np.nan_to_num(x_input, posinf=0, neginf=0)
    
#        # 检查并输出非 float64 数据的坐标
#    for row_index, row in enumerate(x_input):
#        for col_index, value in enumerate(row):
#            if x_input.dtype!= np.float64:
#                print(f"非 float64 数据位于 ({row_index}, {col_index})，值为: {value}")
#    
    #print('x_input',x_input)
    #print('y_output',y_output)
    
    
    val_x_input = data.drop(['SMILES'], axis=1).values
    val_y_output = data['activity'].values
    
    val_x_input = np.nan_to_num(x_input, posinf=0, neginf=0)
    
#    # 查找 x_input 中的无穷值位置
#    infinity_positions_x = np.argwhere(np.isinf(x_input))
#    if len(infinity_positions_x) > 0:
#        column_names = data.drop(['SMILES'], axis=1).columns
#        infinity_column_names_x = [column_names[pos[1]] for pos in infinity_positions_x]
#        print("在 x_input 中的无穷值所在列名：", infinity_column_names_x, '         位置：',infinity_positions_x)
#    else:
#        print("在 x_input 中没有无穷值")
#
#    # 查找 val_x_input 中的无穷值位置
#    infinity_positions_val_x = np.argwhere(np.isinf(val_x_input))
#    if len(infinity_positions_val_x) > 0:
#        column_names = data.drop(['SMILES'], axis=1).columns
#        infinity_column_names_val_x = [column_names[pos[1]] for pos in infinity_positions_val_x]
#        print("在 val_x_input 中的无穷值所在列名：", infinity_column_names_val_x, '        位置：',infinity_positions_x)
#    else:
#        print("在 val_x_input 中没有无穷值")
    
        # 标准化特征（假设特征是数值型的）
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    x_input = scaler.fit_transform(x_input)
    val_x_input = scaler.fit_transform(val_x_input)

    # 数据放缩处理（例如，将特征值缩放到 0 到 1 之间）
    from sklearn.preprocessing import MinMaxScaler
    min_max_scaler = MinMaxScaler(feature_range=(0, 1))
    x_input = min_max_scaler.fit_transform(x_input)
    val_x_input = min_max_scaler.fit_transform(val_x_input)
    
        # 转换为Tensor
    x = torch.Tensor(x_input)
    y_true = torch.Tensor(y_output).view(-1, 1)    
    
    val_x = torch.Tensor(val_x_input)
    val_y_true = torch.Tensor(val_y_output).view(-1, 1)
    
    
    # 数据维度
    input_dim = x.shape[1]
    output_dim = 1
    hidden_dim1 = 256
    hidden_dim2 = 128
    hidden_dim3 = 64
    learning_rate = 0.001
    num_epochs = 100
    weight_decay = 0.001
    # MLP模型定义
    mlp = nn.Sequential(  
        nn.Linear(input_dim, hidden_dim1),  # 第一层隐藏层  
        nn.ReLU(),  
        nn.Linear(hidden_dim1, hidden_dim2),  # 第二层隐藏层  
        nn.ReLU(),  
        nn.Linear(hidden_dim2, hidden_dim3),  # 第三层隐藏层  
        nn.ReLU(),  
        nn.Linear(hidden_dim3, output_dim),  # 输出层  
        nn.Sigmoid()  
    )  
        
    # 优化器和损失函数
    optimizer = torch.optim.Adam(mlp.parameters(), lr=learning_rate,weight_decay=weight_decay)
    #loss_func =  nn.BCEWithLogitsLoss()
    loss_func =  nn.MSELoss()
    best_val_loss = float('inf')
    num_iterations_without_improvement = 0  # Initialize counter for iterations without improvement. 
    
    for epoch in range(num_epochs):
        
        print(f"开始训练第{epoch}组epochs")
        prediction = mlp(x)
        #print('prediction',prediction)
        
        #prediction_np = prediction.detach().numpy()
        #df = pd.DataFrame(prediction_np)
        #df.to_csv('prediction.csv', index=False)
        
        loss = loss_func(prediction, y_true)
        print('loss',loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print(optimizer)
        
        with torch.no_grad():
            val_prediction=mlp(val_x)
            val_loss = loss_func(val_prediction, val_y_true)
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {loss.item()}, Val Loss: {val_loss.item()}")
        
        #print("检查数据和模型输出:")
        #print("是否存在 NaN 或无穷大值在训练预测结果中:", torch.isnan(prediction).any() or torch.isinf(prediction).any())
        #print("是否存在 NaN 或无穷大值在验证预测结果中:", torch.isnan(val_prediction).any() or torch.isinf(val_prediction).any())
        #print("是否存在 NaN 或无穷大值在训练真实值中:", torch.isnan(y_true).any() or torch.isinf(y_true).any())
        #print("是否存在 NaN 或无穷大值在验证真实值中:", torch.isnan(val_y_true).any() or torch.isinf(val_y_true).any())
        #
          
        if val_loss < best_val_loss:
            best_val_loss = val_loss  
            num_iterations_without_improvement = 0
            torch.save(mlp.state_dict(), f"models_{file}/model_{i+1}.pt")
            #torch.save(mlp, "model_saved/mlp1.pt")
            print(f'model_{i}训练结果更新为第{epoch}个模型')
            best_epoch = epoch
        else :
            num_iterations_without_improvement += 1  
            if num_iterations_without_improvement == 300:
                print('300次迭代没有更新，结束迭代')
                print(f'最棒的模型是第{best_epoch}个epoch')
                break
        print('         ')
        

    prediction_np = prediction.data.numpy()
        # 将 prediction 添加到 DataFrame
    #data['prediction'] = prediction_np
        # 将 DataFrame 保存到 CSV 文件
    #data.to_csv('output/output_train.csv', index=True)
    
    # 预测并计算AUC
    prediction_np = prediction.detach().numpy().flatten()
    auc = roc_auc_score(y_true, prediction_np)
    
    threshold = 0.5
    binary_prediction = np.where(prediction_np > threshold, 1, 0)
    
    acc = accuracy_score(y_true, binary_prediction)
    f1 = f1_score(y_true, binary_prediction)
    recall = recall_score(y_true, binary_prediction)
    mcc = matthews_corrcoef(y_true, binary_prediction)
    output_file = "train_scores.csv"
    
    
    auc_scores.append(auc)
    acc_scores.append(acc)
    f1_scores.append(f1)
    recall_scores.append(recall)
    mcc_scores.append(mcc)
    
    print('第',i,'组模型')
    print("AUC:", auc)
    print("ACC:", acc)
    print("F1:", f1)
    print("Recall:", recall)
    print("MCC:", mcc)
    print('    ')
      
    

开始训练第0组epochs
loss tensor(0.2541, grad_fn=<MseLossBackward>)
Epoch 1/100, Train Loss: 0.2541419267654419, Val Loss: 0.23977328836917877
model_0训练结果更新为第0个模型
         
开始训练第1组epochs
loss tensor(0.2398, grad_fn=<MseLossBackward>)
Epoch 2/100, Train Loss: 0.23977328836917877, Val Loss: 0.2253342866897583
model_0训练结果更新为第1个模型
         
开始训练第2组epochs
loss tensor(0.2253, grad_fn=<MseLossBackward>)
Epoch 3/100, Train Loss: 0.2253342866897583, Val Loss: 0.2183333784341812
model_0训练结果更新为第2个模型
         
开始训练第3组epochs
loss tensor(0.2183, grad_fn=<MseLossBackward>)
Epoch 4/100, Train Loss: 0.2183333784341812, Val Loss: 0.22224198281764984
         
开始训练第4组epochs
loss tensor(0.2222, grad_fn=<MseLossBackward>)
Epoch 5/100, Train Loss: 0.22224198281764984, Val Loss: 0.22438795864582062
         
开始训练第5组epochs
loss tensor(0.2244, grad_fn=<MseLossBackward>)
Epoch 6/100, Train Loss: 0.22438795864582062, Val Loss: 0.22177940607070923
         
开始训练第6组epochs
loss tensor(0.2218, grad_fn=<MseLossBackward>)
Ep

loss tensor(0.1158, grad_fn=<MseLossBackward>)
Epoch 62/100, Train Loss: 0.11576076596975327, Val Loss: 0.11159300059080124
model_0训练结果更新为第61个模型
         
开始训练第62组epochs
loss tensor(0.1116, grad_fn=<MseLossBackward>)
Epoch 63/100, Train Loss: 0.11159300059080124, Val Loss: 0.09749735891819
model_0训练结果更新为第62个模型
         
开始训练第63组epochs
loss tensor(0.0975, grad_fn=<MseLossBackward>)
Epoch 64/100, Train Loss: 0.09749735891819, Val Loss: 0.08939655870199203
model_0训练结果更新为第63个模型
         
开始训练第64组epochs
loss tensor(0.0894, grad_fn=<MseLossBackward>)
Epoch 65/100, Train Loss: 0.08939655870199203, Val Loss: 0.08601576089859009
model_0训练结果更新为第64个模型
         
开始训练第65组epochs
loss tensor(0.0860, grad_fn=<MseLossBackward>)
Epoch 66/100, Train Loss: 0.08601576089859009, Val Loss: 0.09407203644514084
         
开始训练第66组epochs
loss tensor(0.0941, grad_fn=<MseLossBackward>)
Epoch 67/100, Train Loss: 0.09407203644514084, Val Loss: 0.1472986340522766
         
开始训练第67组epochs
loss tensor(0.1473, grad_fn=<

model_1训练结果更新为第16个模型
         
开始训练第17组epochs
loss tensor(0.0983, grad_fn=<MseLossBackward>)
Epoch 18/100, Train Loss: 0.09825042635202408, Val Loss: 0.08886905014514923
model_1训练结果更新为第17个模型
         
开始训练第18组epochs
loss tensor(0.0889, grad_fn=<MseLossBackward>)
Epoch 19/100, Train Loss: 0.08886905014514923, Val Loss: 0.0788295567035675
model_1训练结果更新为第18个模型
         
开始训练第19组epochs
loss tensor(0.0788, grad_fn=<MseLossBackward>)
Epoch 20/100, Train Loss: 0.0788295567035675, Val Loss: 0.06887280195951462
model_1训练结果更新为第19个模型
         
开始训练第20组epochs
loss tensor(0.0689, grad_fn=<MseLossBackward>)
Epoch 21/100, Train Loss: 0.06887280195951462, Val Loss: 0.05889429897069931
model_1训练结果更新为第20个模型
         
开始训练第21组epochs
loss tensor(0.0589, grad_fn=<MseLossBackward>)
Epoch 22/100, Train Loss: 0.05889429897069931, Val Loss: 0.04945957288146019
model_1训练结果更新为第21个模型
         
开始训练第22组epochs
loss tensor(0.0495, grad_fn=<MseLossBackward>)
Epoch 23/100, Train Loss: 0.04945957288146019, Val Loss: 0.

Epoch 67/100, Train Loss: 0.005817749537527561, Val Loss: 0.004005169030278921
model_1训练结果更新为第66个模型
         
开始训练第67组epochs
loss tensor(0.0040, grad_fn=<MseLossBackward>)
Epoch 68/100, Train Loss: 0.004005169030278921, Val Loss: 0.004622716922312975
         
开始训练第68组epochs
loss tensor(0.0046, grad_fn=<MseLossBackward>)
Epoch 69/100, Train Loss: 0.004622716922312975, Val Loss: 0.005140708293765783
         
开始训练第69组epochs
loss tensor(0.0051, grad_fn=<MseLossBackward>)
Epoch 70/100, Train Loss: 0.005140708293765783, Val Loss: 0.0038907427806407213
model_1训练结果更新为第69个模型
         
开始训练第70组epochs
loss tensor(0.0039, grad_fn=<MseLossBackward>)
Epoch 71/100, Train Loss: 0.0038907427806407213, Val Loss: 0.004348070826381445
         
开始训练第71组epochs
loss tensor(0.0043, grad_fn=<MseLossBackward>)
Epoch 72/100, Train Loss: 0.004348070826381445, Val Loss: 0.004647604189813137
         
开始训练第72组epochs
loss tensor(0.0046, grad_fn=<MseLossBackward>)
Epoch 73/100, Train Loss: 0.004647604189813137, Va

model_2训练结果更新为第24个模型
         
开始训练第25组epochs
loss tensor(0.0302, grad_fn=<MseLossBackward>)
Epoch 26/100, Train Loss: 0.030227309092879295, Val Loss: 0.02802090346813202
model_2训练结果更新为第25个模型
         
开始训练第26组epochs
loss tensor(0.0280, grad_fn=<MseLossBackward>)
Epoch 27/100, Train Loss: 0.02802090346813202, Val Loss: 0.026532689109444618
model_2训练结果更新为第26个模型
         
开始训练第27组epochs
loss tensor(0.0265, grad_fn=<MseLossBackward>)
Epoch 28/100, Train Loss: 0.026532689109444618, Val Loss: 0.024904638528823853
model_2训练结果更新为第27个模型
         
开始训练第28组epochs
loss tensor(0.0249, grad_fn=<MseLossBackward>)
Epoch 29/100, Train Loss: 0.024904638528823853, Val Loss: 0.0238017775118351
model_2训练结果更新为第28个模型
         
开始训练第29组epochs
loss tensor(0.0238, grad_fn=<MseLossBackward>)
Epoch 30/100, Train Loss: 0.0238017775118351, Val Loss: 0.02240240015089512
model_2训练结果更新为第29个模型
         
开始训练第30组epochs
loss tensor(0.0224, grad_fn=<MseLossBackward>)
Epoch 31/100, Train Loss: 0.02240240015089512, Val Los

loss tensor(0.0049, grad_fn=<MseLossBackward>)
Epoch 89/100, Train Loss: 0.004908788483589888, Val Loss: 0.004082086496055126
model_2训练结果更新为第88个模型
         
开始训练第89组epochs
loss tensor(0.0041, grad_fn=<MseLossBackward>)
Epoch 90/100, Train Loss: 0.004082086496055126, Val Loss: 0.004442066885530949
         
开始训练第90组epochs
loss tensor(0.0044, grad_fn=<MseLossBackward>)
Epoch 91/100, Train Loss: 0.004442066885530949, Val Loss: 0.004588370677083731
         
开始训练第91组epochs
loss tensor(0.0046, grad_fn=<MseLossBackward>)
Epoch 92/100, Train Loss: 0.004588370677083731, Val Loss: 0.003997118677943945
model_2训练结果更新为第91个模型
         
开始训练第92组epochs
loss tensor(0.0040, grad_fn=<MseLossBackward>)
Epoch 93/100, Train Loss: 0.003997118677943945, Val Loss: 0.0043671829625964165
         
开始训练第93组epochs
loss tensor(0.0044, grad_fn=<MseLossBackward>)
Epoch 94/100, Train Loss: 0.0043671829625964165, Val Loss: 0.004343749489635229
         
开始训练第94组epochs
loss tensor(0.0043, grad_fn=<MseLossBackward>)
Epo

model_3训练结果更新为第42个模型
         
开始训练第43组epochs
loss tensor(0.0175, grad_fn=<MseLossBackward>)
Epoch 44/100, Train Loss: 0.01750565879046917, Val Loss: 0.01674840785562992
model_3训练结果更新为第43个模型
         
开始训练第44组epochs
loss tensor(0.0167, grad_fn=<MseLossBackward>)
Epoch 45/100, Train Loss: 0.01674840785562992, Val Loss: 0.01493618544191122
model_3训练结果更新为第44个模型
         
开始训练第45组epochs
loss tensor(0.0149, grad_fn=<MseLossBackward>)
Epoch 46/100, Train Loss: 0.01493618544191122, Val Loss: 0.015402335673570633
         
开始训练第46组epochs
loss tensor(0.0154, grad_fn=<MseLossBackward>)
Epoch 47/100, Train Loss: 0.015402335673570633, Val Loss: 0.013603326864540577
model_3训练结果更新为第46个模型
         
开始训练第47组epochs
loss tensor(0.0136, grad_fn=<MseLossBackward>)
Epoch 48/100, Train Loss: 0.013603326864540577, Val Loss: 0.013816272839903831
         
开始训练第48组epochs
loss tensor(0.0138, grad_fn=<MseLossBackward>)
Epoch 49/100, Train Loss: 0.013816272839903831, Val Loss: 0.013071145862340927
model_3训练结果更新为第

开始训练第0组epochs
loss tensor(0.2434, grad_fn=<MseLossBackward>)
Epoch 1/100, Train Loss: 0.24342194199562073, Val Loss: 0.228919118642807
model_4训练结果更新为第0个模型
         
开始训练第1组epochs
loss tensor(0.2289, grad_fn=<MseLossBackward>)
Epoch 2/100, Train Loss: 0.228919118642807, Val Loss: 0.21233181655406952
model_4训练结果更新为第1个模型
         
开始训练第2组epochs
loss tensor(0.2123, grad_fn=<MseLossBackward>)
Epoch 3/100, Train Loss: 0.21233181655406952, Val Loss: 0.20612682402133942
model_4训练结果更新为第2个模型
         
开始训练第3组epochs
loss tensor(0.2061, grad_fn=<MseLossBackward>)
Epoch 4/100, Train Loss: 0.20612682402133942, Val Loss: 0.20600906014442444
model_4训练结果更新为第3个模型
         
开始训练第4组epochs
loss tensor(0.2060, grad_fn=<MseLossBackward>)
Epoch 5/100, Train Loss: 0.20600906014442444, Val Loss: 0.2022566944360733
model_4训练结果更新为第4个模型
         
开始训练第5组epochs
loss tensor(0.2023, grad_fn=<MseLossBackward>)
Epoch 6/100, Train Loss: 0.2022566944360733, Val Loss: 0.19615380465984344
model_4训练结果更新为第5个模型
         
开始训练

Epoch 57/100, Train Loss: 0.008869988843798637, Val Loss: 0.007739965803921223
model_4训练结果更新为第56个模型
         
开始训练第57组epochs
loss tensor(0.0077, grad_fn=<MseLossBackward>)
Epoch 58/100, Train Loss: 0.007739965803921223, Val Loss: 0.008114921860396862
         
开始训练第58组epochs
loss tensor(0.0081, grad_fn=<MseLossBackward>)
Epoch 59/100, Train Loss: 0.008114921860396862, Val Loss: 0.0073682889342308044
model_4训练结果更新为第58个模型
         
开始训练第59组epochs
loss tensor(0.0074, grad_fn=<MseLossBackward>)
Epoch 60/100, Train Loss: 0.0073682889342308044, Val Loss: 0.007581790909171104
         
开始训练第60组epochs
loss tensor(0.0076, grad_fn=<MseLossBackward>)
Epoch 61/100, Train Loss: 0.007581790909171104, Val Loss: 0.006432114169001579
model_4训练结果更新为第60个模型
         
开始训练第61组epochs
loss tensor(0.0064, grad_fn=<MseLossBackward>)
Epoch 62/100, Train Loss: 0.006432114169001579, Val Loss: 0.006715346593409777
         
开始训练第62组epochs
loss tensor(0.0067, grad_fn=<MseLossBackward>)
Epoch 63/100, Train Loss: 0.0

In [41]:
te_file_prefix = f"{file}_Feature_fusion/data_test"
te_file_extension = ".csv"

import os 

avg_auc = 0.0  
avg_acc = 0.0  
avg_f1 = 0.0  
avg_recall = 0.0  
avg_mcc = 0.0  


#te_file_paths = [te_file_prefix + str(i) + te_file_extension for i in range(start_index, end_index+1)]  
results = pd.DataFrame(columns=['AUC', 'Accuracy', 'F1', 'Recall', 'MCC'])   
    
#for i,te_file_path in enumerate(te_file_paths):  
for i in range(start_index, end_index): 
    te_file_path = te_file_prefix + str(i) + te_file_extension
    data_test = pd.read_csv(te_file_path)  
    if file == 'BBB':
        data_test = data_test.head(129)  

    #print(data_test)
      
    te_x_input = data_test.drop(['SMILES'], axis=1).values  
    #print('1te_x_input.shape',te_x_input.shape)
    
    te_y_true = data_test['activity'].values  
    te_x_input = te_x_input.astype(np.float64)
    #print('2te_x_input.shape',te_x_input.shape)
    
    te_x_input = np.nan_to_num(te_x_input, posinf=0, neginf=0)
    #print('3te_x_input',te_x_input.shape)    
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    te_x_input = scaler.fit_transform(te_x_input)

    # 数据放缩处理（例如，将特征值缩放到 0 到 1 之间）
    from sklearn.preprocessing import MinMaxScaler
    min_max_scaler = MinMaxScaler(feature_range=(0, 1))
    te_x_input = min_max_scaler.fit_transform(te_x_input)
    te_x_input = torch.Tensor(te_x_input)
    #print('4te_x_input.shape',te_x_input.shape)
    mlp.load_state_dict(torch.load(f"models_{file}/model_{i+1}.pt")) 
    
    #print('39 te_x_input te_x_input ',te_x_input)
    
    #print("Length of te_prediction:", len(te_prediction))
    #print("Length of data_test:", len(data_test))
    #print("Length of data index:", len(data.index))
    #
    with torch.no_grad():    
        
       # print("47  te_x_input  te_x_input",te_x_input)
        
        te_prediction = mlp(te_x_input)  
        
        ############################################################
        te_prediction = te_prediction.numpy()
        data = pd.DataFrame(te_prediction)
        data.to_csv(f'output1/test_prediction{i+1}.csv', mode='a', index=False)
        ############################################################
        
        #print('te_y_true',len(te_y_true))
        #print('te_prediction',len(te_prediction))
        #print('te_y_true',te_y_true)
        #print('te_prediction',te_prediction)
        #print('检查 te_y_true 是否包含 NaN',np.isnan(te_y_true).any())  # 检查 te_y_true 是否包含 NaN
        #print('检查 te_y_true 是否包含无穷值',np.isinf(te_y_true).any())  # 检查 te_y_true 是否包含无穷值
        #print('检查 te_prediction 是否包含 NaN',np.isnan(te_prediction).any())  # 检查 te_prediction 是否包含 NaN
        #print('检查 te_prediction 是否包含无穷值',np.isinf(te_prediction).any())  # 检查 te_prediction 是否包含无穷值
        #
        auc = roc_auc_score(te_y_true, te_prediction)
        #print(auc)
        
        #print('54  te_predictionte_prediction',te_prediction)    
        #te_prediction = te_prediction.numpy()      
        data_test[f'te_prediction_{i+1}'] = te_prediction 
        
        auc = roc_auc_score(te_y_true, te_prediction)
        #print(auc)
        
        #data_test.to_csv(f'output1/te_prediction{i+1}.csv', mode='a', index=True)
        #print('te_prediction  ',te_prediction)
        
        auc = roc_auc_score(te_y_true, te_prediction)  
        print(f'AUC_{i+1}: {auc}')
        
        te_prediction_binary = np.where(te_prediction > 0.5, 1, 0)  
        acc = accuracy_score(te_y_true, te_prediction_binary)  
        print(f'Accuracy_{i+1}: {acc}')
        f1 = f1_score(te_y_true, te_prediction_binary)
        recall = recall_score(te_y_true, te_prediction_binary)
        mcc = matthews_corrcoef(te_y_true, te_prediction_binary)
        print(f'f1_{i+1}: {f1}')
        print(f'recall_{i+1}: {recall}')
        print(f'mcc_{i+1}: {mcc}')
        print('             ')
        print('             ')
        
        avg_auc += auc  
        avg_acc += acc  
        avg_f1 += f1  
        avg_recall += recall  
        avg_mcc += mcc
        
        results = results.append({'AUC': auc, 'Accuracy': acc, 'F1': f1, 'Recall': recall, 'MCC': mcc}, ignore_index=True) 
results.to_csv('测试集预测_results.csv', index=False)

avg_auc /= len(range(start_index, end_index)) 
avg_acc /= len(range(start_index, end_index)) 
avg_f1 /= len(range(start_index, end_index))  
avg_recall /= len(range(start_index, end_index))
avg_mcc /= len(range(start_index, end_index))

auc_te_values = results['AUC'].values
accuracy_te_values = results['Accuracy'].values
f1_te_values = results['F1'].values
recall_te_values = results['Recall'].values
mcc_te_values = results['MCC'].values

std_auc_te = np.std(auc_te_values)
std_acc_te = np.std(accuracy_te_values)
std_f1_te = np.std(f1_te_values)
std_recall_te = np.std(recall_te_values)
std_mcc_te = np.std(mcc_te_values)

print(f'Average AUC: {avg_auc:.6f}±{std_auc_te:.6f}')
print(f'Average Accuracy: {avg_acc:.6f}±{std_acc_te:.6f}')
print(f'Average F1: {avg_f1:.6f}±{std_f1_te:.6f}')
print(f'Average Recall: {avg_recall:.6f}±{std_recall_te:.6f}')
print(f'Average MCC: {avg_mcc:.6f}±{std_mcc_te:.6f}')

AUC_1: 0.8813291139240507
Accuracy_1: 0.8932038834951457
f1_1: 0.744186046511628
recall_1: 0.6666666666666666
mcc_1: 0.6852367480867336
             
             
AUC_2: 0.8583333333333334
Accuracy_2: 0.7647058823529411
f1_2: 0.6666666666666666
recall_2: 0.8
mcc_2: 0.5092010548749033
             
             
AUC_3: 0.8571428571428571
Accuracy_3: 0.7843137254901961
f1_3: 0.6666666666666667
recall_3: 0.6875
mcc_3: 0.5079721589671172
             
             
AUC_4: 0.8129401408450704
Accuracy_4: 0.7475728155339806
f1_4: 0.6486486486486486
recall_4: 0.75
mcc_4: 0.46753634073087225
             
             
Average AUC: 0.852436±0.024757
Average Accuracy: 0.797449±0.056792
Average F1: 0.681542±0.036908
Average Recall: 0.726042±0.052571
Average MCC: 0.542487±0.084105


In [39]:
learning_rate = 0.1
num_epochs = 100
weight_decay = 0.01




learning_rate = 0.01
num_epochs = 100
weight_decay = 0.01

Average AUC: 0.943448±0.020126
Average Accuracy: 0.796512±0.098723
Average F1: 0.831558±0.094451
Average Recall: 0.774691±0.173856
Average MCC: 0.624705±0.130822





    output_dim = 1
    hidden_dim1 = 512
    hidden_dim2 = 256
    hidden_dim3 = 64
    learning_rate = 0.001
    num_epochs = 100
    weight_decay = 0.01
    
Average AUC: 0.725601±0.064932
Average Accuracy: 0.598296±0.056470
Average F1: 0.666462±0.026149
Average Recall: 0.785408±0.030126
Average MCC: 0.353964±0.151104

SyntaxError: invalid syntax (830061736.py, line 12)